# Typical Start

`nbdev` needs ot know where to export the module code fomr this notebook. You will typically just export to a module name the same as the notebook but you can rename and add extra directories to create a package structure if you wish

In [1]:
# hide
# default_exp hello_sciflow

The below code is an example of how to define a step. A step is a function we chain together to form a workflow. Typically they will have inputs and return values which can be used in subsequent steps but we start simple here.

## Breaking down the named step directive:

* `exportn_step:first` = export a named step called **first**

In [2]:
# exportn_step:first


def first():
    print("The first step")

Auto-reloading modules is very useful when using `nbdev` as changes to underlying modules are picked up without having to restart the kernel.

In [3]:
%load_ext autoreload
%autoreload 2

You need to export all code that is gonig to be used in flows - this means things like import statements as well as function definitions.

In [4]:
# export

# Params

* `sciflow` uses the papermill format for paramaeterising notebooks. 
* See here for how to specify papermill params: https://papermill.readthedocs.io/en/latest/usage-parameterize.html. 
* These parameters will be available to use or edit in your flows.

In [5]:
# export

traffic_percent = 1

In [6]:
# exportn_step:second


def second(traffic_percent):
    results = {}
    # Currently step functions need to return a dictionary. The keys: artifacts and metrics are reserved for experiment tracking otherwiose any other named keys will be available in future steps.
    results["traffic_multiplied"] = traffic_percent * 2
    return results

# See the results

## 1. Command Line Interface

# 1.1  Convert the notebooks to sciflow modules

In [11]:
# tst
!sciflow_build_lib

Converted complete_workflow.ipynb.
Converted experiment_tracking.ipynb.
Converted hello_sciflow.ipynb.
Converted running_metaflows.ipynb.
Converted why_make.ipynb.


# 1.2 Tests that the notebooks work!

In [13]:
# tst
!nbdev_test_nbs --pause=1

testing /home/jovyan/git/sciflow/examples/complete_workflow.ipynb
testing /home/jovyan/git/sciflow/examples/experiment_tracking.ipynb
testing /home/jovyan/git/sciflow/examples/hello_sciflow.ipynb
testing /home/jovyan/git/sciflow/examples/running_metaflows.ipynb
testing /home/jovyan/git/sciflow/examples/why_make.ipynb
All tests are passing!


# 1.3 See the created `Flow`

In [14]:
# tst
!cat hello_sciflow.py

#!/usr/bin/env python
# coding=utf-8
# SCIFLOW GENERATED FILE - EDIT COMPANION NOTEBOOK
from metaflow import FlowSpec, step, current, Parameter
from sciflow_examples.hello_sciflow import first, second
from sciflow_examples.hello_sciflow import traffic_percent
from sacred import Experiment
from sciflow.experiment.lake_observer import AWSLakeObserver
import time

ex = Experiment("hello_sciflow")
# TODO inject observers
obs = AWSLakeObserver(project="sciflow_examples", experiment_name="hello_sciflow")
ex.observers.append(obs)

@ex.config
def config():
    flow_run_id = None
    artifacts = []
    metrics = []
    

class HelloSciflowFlow(FlowSpec):
    traffic_percent = Parameter('traffic_percent', default=traffic_percent)
    artifacts = []
    metrics = []

    @step
    def start(self):
        first()
        self.start_time = time.time()
        self.next(self.second)

    @step
    def second(self):
        results = second(self.traffic_percent)

        for key in results.keys():
 

# Run a `Flow`, this example uses `metaflow`

In [15]:
##tst
!source ~/.sciflow/env && python hello_sciflow.py --no-pylint run

Metaflow 2.3.4 executing HelloSciflowFlow for user:Donal
Validating your flow...
    The graph looks good!
2021-09-08 14:30:18.293 Workflow starting (run-id 1631111418290567):
2021-09-08 14:30:18.296 [1631111418290567/start/1 (pid 15689)] Task is starting.
2021-09-08 14:30:19.319 [1631111418290567/start/1 (pid 15689)] The first step
2021-09-08 14:30:19.444 [1631111418290567/start/1 (pid 15689)] Task finished successfully.
2021-09-08 14:30:19.449 [1631111418290567/second/2 (pid 15722)] Task is starting.
2021-09-08 14:30:20.612 [1631111418290567/second/2 (pid 15722)] Task finished successfully.
2021-09-08 14:30:20.617 [1631111418290567/end/3 (pid 15755)] Task is starting.
2021-09-08 14:30:21.646 [1631111418290567/end/3 (pid 15755)] INFO - hello_sciflow - Running command 'track_flow'
2021-09-08 14:30:22.078 [1631111418290567/end/3 (pid 15755)] INFO - hello_sciflow - Started
2021-09-08 14:30:22.079 [1631111418290567/end/3 (pid 15755)] INFO - hello_sciflow - Completed after 0:00:00
2021-09-

# 2. Programmatically

TBC..